# Malware Analysis & Triage Kit
This notebook performs the initial stages of immediate malware triage.

## How To
Take your malware specimen and drop it into the `dropbox` directory. The notebook will walk you through the stages of initial analysis.

At the end of this process, you will have a triage report in the `saved-specimens` diretory. This report includes findings from initial triage, including the defanged specimen in a password-proteced Zip file and static analysis artifacts.

# Imports and Setup

In [ ]:
# Imports
from hashlib import *
import sys
import os
from getpass import getpass
from virus_total_apis import PublicApi as VirusTotalPublicApi
import json
from MalwareSample import *
from pprint import pprint
import os.path
from time import sleep

### Check Dropbox and Saved-Specimens

In [ ]:
MalwareSample.check_dir("dropbox")
MalwareSample.check_dir("saved-specimens")
empty = MalwareSample.is_dir_empty("dropbox")
if empty:
    print(r"  \\--> " + recc + "Put some samples in the dropbox!")

### Enumerate Samples in the Dropbox

In [ ]:
samples=!ls dropbox/*
for s in samples:
    print(info + "Sample: " + s)

In [ ]:
sample_obj = [MalwareSample(s) for s in samples]

### Create a Saved Specimen directory for the specimen(s)

In [ ]:
for obj in sample_obj:
    saved_sample_name = MalwareSample.create_specimen_dirs(obj.sample_name)
    obj.saved_sample_name = saved_sample_name

### Defang Sample

In [ ]:
for obj in sample_obj:
    sample_path = MalwareSample.move_and_defang(obj.sample_name, obj.saved_sample_name)
    obj.sample_path = sample_path

---

## File Hashes

### SHA256 Sum

In [ ]:
for obj in sample_obj:
    hash = MalwareSample.get_sha256sum(obj.sample_path, obj.saved_sample_name)
    obj.sha256sum = hash
    print(info + obj.sample_name + ": " + obj.sha256sum)

---

## String Analysis

### StringSifter
StringSifter is a FLARE developed tool that uses an ML model to rank a binary's strings by relevance to malware analysis.

In [ ]:
length = int(input(recc + "Input your desired minimum string length [default is 4, 6-8 is recommended] > "))

In [ ]:
for obj in sample_obj:
    MalwareSample.pull_strings(length, obj.saved_sample_name, obj.sample_path)

## VT Analysis
Submit samples to Virus Total and generate a malicious confidence level.

In [ ]:
VT_API_KEY = getpass("Enter VirusTotal API Key (blank if none): ")

In [ ]:
if VT_API_KEY:
    vt = VirusTotalPublicApi(VT_API_KEY)
else:
    print(info + "No VT API Key. Skipping...")

Note: If there are more than 4 samples in the dropbox, hashes are submitted with a sleep of 16 seconds to remain under the public API rate limit. So hit go, grab a beverage of choice, stretch out and relax. This could be a while depending on how many samples you're submitting.

In [ ]:
if VT_API_KEY:
    for obj in sample_obj:
        print(info + obj.sample_name + ":")
        print(r"  \\--> " + info + "SHA256sum: " + obj.sha256sum)
        res = vt.get_file_report(obj.sha256sum)
        conf = malicious_confidence(res)
        print(r"  \\--> " + info + "Confidence level: " + str(conf))
        crit_level =  determine_criticality(conf)
        obj.criticality = crit_level
        

        if len(sample_obj) >= 5:
            sleep(16)
        
else:
    print(info + "No VT API Key. Skipping...")

## Zip and Password Protect

In [ ]:
for obj in sample_obj:
    zip_file = MalwareSample.zip_and_password_protect(obj.sample_path, obj.saved_sample_name)
    MalwareSample.delete_unzipped_sample(obj.sample_path, zip_file)

---

### Debug Object Vars

In [ ]:
for obj in sample_obj:
    pprint(vars(obj))